In [1]:
! pip install selenium
! pip install pandas



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time

# List of Thai dishes
thai_dishes = ["Pad Thai", "Tom Yum Soup", "Massaman Curry", "Green Curry", "Som Tum"]

# Set the path to your ChromeDriver executable
driver_path = "chromedriver.exe"
URL = 'https://unsplash.com/'

# Create a Service object
service = Service(driver_path)

# Initialize the WebDriver with the Service object
driver = webdriver.Chrome(service=service)

# Create a base directory to save the images
base_save_dir = "thai_dishes_images"
if not os.path.exists(base_save_dir):
    os.makedirs(base_save_dir)

for dish in thai_dishes:
    # Open Unsplash website and search for the dish
    driver.get(URL)
    driver.maximize_window()
    time.sleep(5)  # Allow the page to load

    # Locate the search bar using its attributes
    search_bar = driver.find_element(By.XPATH, "//input[@type='search']")
    search_bar.clear()  # Clear the search bar
    search_bar.send_keys(f"{dish} dish images")
    search_bar.send_keys(Keys.ENTER)
    time.sleep(5)  # Wait for the search results to load

    # Scroll the page to load more images (important for lazy loading)
    for _ in range(5):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)

    # Get all image tags
    images_tags = driver.find_elements(By.XPATH, "//img")
    print(f"Found {len(images_tags)} image tags for {dish}.")

    # Extract image URLs (checking src, data-src, and srcset attributes)
    images_urls = [img.get_attribute('src') for img in images_tags if img.get_attribute('src') and 'https' in img.get_attribute('src')]
    images_urls += [img.get_attribute('data-src') for img in images_tags if img.get_attribute('data-src') and 'https' in img.get_attribute('data-src')]
    images_urls += [img.get_attribute('srcset') for img in images_tags if img.get_attribute('srcset') and 'https' in img.get_attribute('srcset')]

    # Remove duplicates if any
    images_urls = list(set(images_urls))

    # Create a directory for each dish
    dish_save_dir = os.path.join(base_save_dir, dish.replace(" ", "_"))
    if not os.path.exists(dish_save_dir):
        os.makedirs(dish_save_dir)

    # Download and save the first 4 images
    for i, url in enumerate(images_urls[:30]):
        try:
            # Send HTTP request to fetch the image
            response = requests.get(url, stream=True)

            # Check if the request was successful
            if response.status_code == 200:
                # Get the image's content type (to save with the correct extension)
                content_type = response.headers['Content-Type']
                ext = content_type.split('/')[1]  # e.g., 'jpeg', 'png', etc.

                # Save the image with a unique name
                image_path = os.path.join(dish_save_dir, f"{dish.replace(' ', '_')}_image_{i+1}.{ext}")

                # Open the file in write-binary mode and save the image
                with open(image_path, 'wb') as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)
                print(f"Downloaded {url} as {image_path}")
            else:
                print(f"Failed to download {url}")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

# Optionally: Close the browser after the task is done
driver.quit()
